In [3]:
pip install pandas openpyxl


In [2]:
from pathlib import Path
import pandas as pd
import re
import math
import numpy as np

# === CONFIG ===
SOURCE_PATH = Path(r"C:\Users\ronan.gaborit\OLYMPIQUE DE MARSEILLE\Centre de Formation - Sportif\DEPOT FICHIERS GPS CDF\MATCHS\DB_MATCHS.csv")
OUTPUT_PATH = SOURCE_PATH.with_name("REF_min70or45_REFtables.xlsx")

# Seuils
PRIMARY_THRESHOLD = 70   # minutes
FALLBACK_THRESHOLD = 45  # minutes si aucun match >= 70

# Colonnes candidates
PLAYER_COLS = ["Player Display Name"]
MATCH_COLS  = ["match_id", "Match ID", "Session Title"]
DATE_COLS   = ["Session Date", "Date"]

# Métriques
SUM_COLS = [
    "Total Distance",
    "Sprint Distance",
    "High Speed Running (Absolute)",
    "Entries Zone 5 (Absolute)", "Entries Zone 6 (Absolute)",
    "Accelerations (Absolute)", "Decelerations (Absolute)",
    "Distance Zone 6 (Absolute)"
]
AVG_COLS = ["Distance Per Min"]
MAX_COLS = ["Max Speed"]

# Mapping optionnel d'alias : "nom nettoyé" -> "nom canonique"
# Laisser vide si tu n'en as pas besoin, ou complète-le par ex :
# ALIAS_MAP = {
#     "Kelian Le Pironnec": "Kelian Le Pironnec",
#     "K. Le Pironnec": "Kelian Le Pironnec",
# }
ALIAS_MAP = {}


# --- Utils ---
def _pick_first_existing(df, candidates):
    norm = {c.lower(): c for c in df.columns}
    for c in candidates:
        if c.lower() in norm:
            return norm[c.lower()]
    return None


_HMS_RE = re.compile(r"^\s*(\d{1,2}):([0-5]?\d):([0-5]?\d)(?:\.\d+)?\s*$")
_HM_RE  = re.compile(r"^\s*(\d{1,2}):([0-5]?\d)(?:\.\d+)?\s*$")


def _to_seconds_total_time(x):
    """Parse 'Total Time' -> secondes (hh:mm:ss, mm:ss, min, sec, fraction de jour Excel)."""
    if x is None or (isinstance(x, float) and math.isnan(x)):
        return None
    if isinstance(x, (int, float)):
        v = float(x)
        if 0 < v < 1:   # fraction de jour Excel
            return v * 24 * 3600
        if v > 180:     # déjà des secondes
            return v
        return v * 60   # minutes
    s = str(x).strip().lower().replace(",", ".")
    if not s:
        return None
    m = _HMS_RE.match(s)
    if m:
        h, mm, ss = map(int, m.groups())
        return h*3600 + mm*60 + ss
    m = _HM_RE.match(s)
    if m:
        mm, ss = map(int, m.groups())
        return mm*60 + ss
    m = re.search(r"^(\d+(?:\.\d+)?)\s*(min|mins|m)\b", s)
    if m:
        return float(m.group(1)) * 60.0
    m = re.search(r"^(\d+(?:\.\d+)?)\s*(sec|secs|s|secondes?)\b", s)
    if m:
        return float(m.group(1))
    s_unitless = re.sub(r"\b(min|mins|minutes|sec|secs|seconds|s)\b", "", s).strip()
    try:
        v = float(s_unitless)
        if 0 < v < 1: return v * 24 * 3600
        if v > 180:   return v
        return v * 60
    except Exception:
        return None


def clean_name(s):
    """Nettoie les noms : minuscules, suppression des doublons consécutifs, recapitalisation."""
    if pd.isna(s):
        return s
    # minuscules, remplacement des ?, trim, réduction des espaces
    s = str(s).replace("?", "e").strip().lower()
    s = re.sub(r"\s+", " ", s)

    # suppression des mots consécutifs identiques : "kelian kelian le pironnec" -> "kelian le pironnec"
    tokens = s.split(" ")
    dedup_tokens = []
    for tok in tokens:
        if not dedup_tokens or dedup_tokens[-1] != tok:
            dedup_tokens.append(tok)

    def cap_token(tok):
        tok = "-".join(p.capitalize() for p in tok.split("-"))
        tok = "'".join(p.capitalize() for p in tok.split("'"))
        return tok

    return " ".join(cap_token(t) for t in dedup_tokens)


def apply_alias(name):
    """Applique le mapping d'alias éventuel après nettoyage."""
    if pd.isna(name):
        return name
    return ALIAS_MAP.get(name, name)


def coerce_numeric(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """Force les colonnes à être numériques (gère la virgule décimale FR)."""
    for c in cols:
        if c in df.columns:
            s = (
                df[c]
                .astype("string")
                .str.replace("\u00A0", " ", regex=False)  # espace insécable
                .str.strip()
                .str.replace(",", ".", regex=False)       # virgule -> point
            )
            df[c] = pd.to_numeric(s, errors="coerce")
    return df


def try_read_csv(path: Path):
    """Lecture robuste : essaie plusieurs encodages et séparateurs. Retourne (df, params)."""
    trials = [
        dict(sep=None, engine="python", encoding="utf-8-sig"),
        dict(sep=None, engine="python", encoding="cp1252"),
        dict(sep=None, engine="python", encoding="latin-1"),
        dict(sep=None, engine="python", encoding="utf-16"),
        dict(sep=";", engine="python", encoding="cp1252"),
        dict(sep=",", engine="python", encoding="cp1252"),
    ]
    last_err = None
    for kw in trials:
        try:
            df = pd.read_csv(path, on_bad_lines="skip", **kw)
            return df, kw
        except Exception as e:
            last_err = e
    raise last_err if last_err else RuntimeError("Impossible de lire le CSV.")


def compute_percentiles(df: pd.DataFrame, metrics: list[str], q=(0.10,0.25,0.50,0.75,0.90)):
    """Renvoie un DataFrame de percentiles globaux pour les colonnes numériques présentes."""
    data = []
    for c in metrics:
        if c in df.columns:
            s = pd.to_numeric(df[c], errors="coerce").dropna()
            if not s.empty:
                vals = np.quantile(s, q)
                row = {"Metric": c}
                for qi, v in zip(q, vals):
                    row[f"P{int(qi*100)}"] = round(float(v), 2)
                data.append(row)
    return pd.DataFrame(data)


def main():
    # --- Lecture ---
    df, read_params = try_read_csv(SOURCE_PATH)
    print("Paramètres de lecture utilisés :", read_params)

    player_col = _pick_first_existing(df, PLAYER_COLS)
    match_col  = _pick_first_existing(df, MATCH_COLS) or "Session Title"
    date_col   = _pick_first_existing(df, DATE_COLS)

    if not player_col:
        raise ValueError("Colonne joueur introuvable (Player Display Name).")
    if "Total Time" not in df.columns:
        raise ValueError("La colonne 'Total Time' est absente. Version 'Total Time only' => arrêt.")

    # Nettoyage joueurs + alias
    df[player_col] = df[player_col].map(clean_name).map(apply_alias)

    # Total Time -> minutes
    df["_sec"] = df["Total Time"].apply(_to_seconds_total_time)
    df["Minutes Played"] = (pd.to_numeric(df["_sec"], errors="coerce") / 60.0).round(2)

    # Numériser métriques
    numeric_targets = list({*SUM_COLS, *AVG_COLS, *MAX_COLS})
    df = coerce_numeric(df, numeric_targets)

    # --- Agrégat joueur × match (somme des drills) ---
    agg = {}
    for c in SUM_COLS:
        if c in df.columns:
            agg[c] = "sum"
    for c in AVG_COLS:
        if c in df.columns:
            agg[c] = "mean"
    for c in MAX_COLS:
        if c in df.columns:
            agg[c] = "max"
    agg["_sec"] = "sum"
    if date_col:
        agg[date_col] = "first"

    group_cols = [player_col, match_col]
    mp = df.groupby(group_cols, dropna=False).agg(agg).reset_index()

    # Minutes + DPM recomp
    mp["Minutes Played"] = (mp["_sec"] / 60.0).round(2)
    mp.drop(columns=["_sec"], inplace=True)
    if "Total Distance" in mp.columns:
        td = pd.to_numeric(mp["Total Distance"], errors="coerce")
        mp["Distance Per Min (recomp)"] = (
            td / pd.to_numeric(mp["Minutes Played"], errors="coerce")
        ).where((pd.to_numeric(mp["Minutes Played"], errors="coerce") > 0), pd.NA).round(2)

    # --- Règle 70 / 45 : on garde tous les matchs éligibles, seuil au niveau joueur ---
    eligible_list = []
    thresh_rows = []
    for player, g in mp.groupby(player_col, dropna=False):
        g = g.copy()
        minutes = pd.to_numeric(g["Minutes Played"], errors="coerce")
        total_matches = len(g)
        used_threshold = PRIMARY_THRESHOLD if minutes.max(skipna=True) >= PRIMARY_THRESHOLD else FALLBACK_THRESHOLD
        g_elig = g[minutes >= used_threshold]
        if not g_elig.empty:
            g_elig["Minutes Threshold Used"] = used_threshold
            eligible_list.append(g_elig)
        thresh_rows.append({
            "Player": player,
            "Minutes Threshold Used": used_threshold,
            "Matches Eligible": int(len(g_elig)),
            "Matches Total": int(total_matches)
        })

    eligible_matches = pd.concat(eligible_list, ignore_index=True) if eligible_list else mp.iloc[0:0].copy()
    ref_thresholds = pd.DataFrame(thresh_rows).sort_values("Player")

    # --- REF par joueur (moyennes / max sur TOUS les matchs éligibles) ---
    ref_agg = {}
    for c in SUM_COLS + AVG_COLS:
        if c in eligible_matches.columns:
            ref_agg[c] = "mean"
    for c in MAX_COLS:
        if c in eligible_matches.columns:
            ref_agg[c] = "max"
    ref_agg["Minutes Played"] = "mean"

    if not eligible_matches.empty:
        ref_player = (
            eligible_matches
            .groupby(player_col, dropna=False)
            .agg(ref_agg)
            .rename(columns=lambda c: (
                f"{c} (avg eligible)" if c in (set(SUM_COLS) | set(AVG_COLS) | {"Minutes Played"}) else
                f"{c} (max eligible)" if c in set(MAX_COLS) else c
            ))
            .reset_index()
        )
        # Ajouter le nombre de matchs éligibles
        counts = eligible_matches.groupby(player_col, dropna=False).size().rename("Matches Eligible")
        ref_player = ref_player.merge(counts.reset_index(), on=player_col, how="left")
        # Arrondis
        for col in ref_player.columns:
            if col != player_col and ref_player[col].dtype.kind in "fc":
                ref_player[col] = ref_player[col].round(2)
    else:
        ref_player = pd.DataFrame(columns=[player_col, "Matches Eligible"])

    # --- Percentiles globaux sur les matchs éligibles (pour benchmark) ---
    metric_cols_present = [c for c in (SUM_COLS + AVG_COLS + MAX_COLS + ["Minutes Played"]) if c in eligible_matches.columns]
    ref_percentiles = compute_percentiles(eligible_matches, metric_cols_present, q=(0.10,0.25,0.50,0.75,0.90))

    # --- Export Excel ---
    with pd.ExcelWriter(OUTPUT_PATH, engine="openpyxl") as w:
        mp.to_excel(w, sheet_name="match_player_agg", index=False)
        eligible_matches.to_excel(w, sheet_name="eligible_matches", index=False)
        ref_player.to_excel(w, sheet_name="REF_player_summary", index=False)
        ref_thresholds.to_excel(w, sheet_name="REF_thresholds", index=False)
        ref_percentiles.to_excel(w, sheet_name="REF_metric_percentiles", index=False)

    # Logs console
    print(f"✅ Exporté : {OUTPUT_PATH}")
    print(f"- match_player_agg : {len(mp)} lignes")
    print(f"- eligible_matches : {len(eligible_matches)} lignes")
    print(f"- REF_player_summary : {len(ref_player)} joueurs")
    print(f"- REF_thresholds : {len(ref_thresholds)} joueurs")
    print(f"- REF_metric_percentiles : {len(ref_percentiles)} métriques avec percentiles")


if __name__ == "__main__":
    main()


Paramètres de lecture utilisés : {'sep': None, 'engine': 'python', 'encoding': 'cp1252'}


C:\Users\ronan.gaborit\AppData\Local\Temp\ipykernel_5128\3079282291.py:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g_elig["Minutes Threshold Used"] = used_threshold
C:\Users\ronan.gaborit\AppData\Local\Temp\ipykernel_5128\3079282291.py:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g_elig["Minutes Threshold Used"] = used_threshold
C:\Users\ronan.gaborit\AppData\Local\Temp\ipykernel_5128\3079282291.py:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


✅ Exporté : C:\Users\ronan.gaborit\OLYMPIQUE DE MARSEILLE\Centre de Formation - Sportif\DEPOT FICHIERS GPS CDF\MATCHS\REF_min70or45_REFtables.xlsx
- match_player_agg : 751 lignes
- eligible_matches : 504 lignes
- REF_player_summary : 112 joueurs
- REF_thresholds : 120 joueurs
- REF_metric_percentiles : 11 métriques avec percentiles
